### we need tensorflow and keras at least

In [ ]:
# !pip install tensorflow
# !pip install keras

In [114]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../")

from src.enzyme_hackathon.utils import (one_hot_dict, one_hot_encode, one_hot_encode_screening_data,
                                        one_hot_encode_sequences)
from src.enzyme_hackathon.utils import AA_LABELS

In [115]:
from src.enzyme_hackathon.utils import (create_fc_model, predict, generate_variant, r_squared, nan_mse)

In [5]:
regression_df = pd.read_csv("../data/raw/challenge-0.csv")
df1 = pd.read_csv("../data/raw/challenge-1.csv")
df2 = pd.read_csv("../data/raw/challenge-2.csv")
print(regression_df.describe(), df1.describe(), df2.describe())

        Unnamed: 0  performance    stability  productivity      fitness
count  7000.000000  6896.000000  6878.000000   7000.000000  6878.000000
mean   4010.524714     0.984543     0.781275      1.215665     0.898539
std    2299.783102     0.283531     0.324650      0.462164     0.491000
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%    2026.750000     0.911732     0.499931      0.992109     0.494338
50%    4000.500000     0.997816     0.841235      1.016482     0.891317
75%    5999.250000     1.169030     1.000279      1.501715     1.266698
max    7999.000000     2.253050     2.493611      3.035927     1.940561         Unnamed: 0  performance    stability  productivity      fitness
count  9906.000000  9838.000000  9719.000000   9906.000000  9906.000000
mean   5001.255704     1.077744     1.093033      1.173327     1.330397
std    2884.806712     0.331799     0.317386      0.356221     0.515413
min       1.000000     0.000000     0.000000      0.000000     0

In [97]:
from collections import OrderedDict
encoded_split_dict = one_hot_encode_screening_data(regression_df)
x_train, x_test = (encoded_split_dict['x'][0], encoded_split_dict['x'][1])
y_keys = [key for key in encoded_split_dict['y'].keys()]

y_train, y_test = (OrderedDict(), OrderedDict())
for y_key in y_keys:
    y_train[y_key], y_test[y_key] = (encoded_split_dict['y'][y_key]['data'][0], encoded_split_dict['y'][y_key]['data'][1])

## Regression for the challenge 0

In [116]:
from keras import backend as K
from keras import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2

def create_fc_model(data: dict):
    """Create a multi-task fully connected neural network model

    :param data:
    :return:
    """
    sequence_input = Input(shape=data['x'][0][0].shape, name='sequence')
    sequence = Flatten()(sequence_input)

    sequence = Dense(50, activation='relu', kernel_regularizer=l2(1.1))(sequence)
    sequence = Dropout(.1)(sequence)

    outputs = []
    for task, _ in data['y'].items():
        t = Dense(20, activation='relu', kernel_regularizer=l2(1.1))(sequence)
        t = Dropout(.1)(t)
        t = BatchNormalization()(t)
        outputs.append(Dense(1, name=task)(t))

    model = Model(inputs=[sequence_input], outputs=outputs)
    model.compile(loss=nan_mse, optimizer=Adam(lr=.0001), metrics=[r_squared])
    return model

In [117]:
fc_nn = create_fc_model(encoded_split_dict)
# for key in y_test.keys():
#     y_train_tuples = [(x.tolist()[0], y.tolist()[0], z.tolist()[0]) for x, y, z in zip(y_train[y_keys[0]], y_train[y_keys[1]], y_train[y_keys[2]])]

In [ ]:
fc_nn.fit(x=x_train, y=[y_train[y_keys[0]], y_train[y_keys[1]], y_train[y_keys[2]]],
         epochs=50, batch_size=100)

Epoch 1/50
5250/5250 [==============================] - 1s 135us/step - loss: 47.4171 - productivity_loss: 0.3475 - performance_loss: 0.2375 - stability_loss: 0.2815 - productivity_r_squared: 0.6536 - performance_r_squared: 0.7527 - stability_r_squared: 0.7160
Epoch 2/50
5250/5250 [==============================] - 1s 144us/step - loss: 44.7153 - productivity_loss: 0.3351 - performance_loss: 0.2310 - stability_loss: 0.2580 - productivity_r_squared: 0.6650 - performance_r_squared: 0.7608 - stability_r_squared: 0.7384
Epoch 3/50
5250/5250 [==============================] - 1s 136us/step - loss: 42.1702 - productivity_loss: 0.3298 - performance_loss: 0.2191 - stability_loss: 0.2543 - productivity_r_squared: 0.6701 - performance_r_squared: 0.7739 - stability_r_squared: 0.7415
Epoch 4/50
5250/5250 [==============================] - 1s 128us/step - loss: 39.7541 - productivity_loss: 0.3136 - performance_loss: 0.2207 - stability_loss: 0.2486 - productivity_r_squared: 0.6850 - performance_r_sq

In [120]:
?fc_nn.fit

Signature:
fc_nn.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose=1,
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    **kwargs,
)
Docstring:
Trains the model for a given number of epochs (iterations on a dataset).

# Arguments
    x: Numpy array of training data (if the model has a single input),
        or list of Numpy arrays (if the model has multiple inputs).
        If input layers in the model are named, you can also pass a
        dictionary mapping input names to Numpy arrays.
        `x` can be `None` (default) if feeding from
        framework-native tensors (e.g. TensorFlow data tensors).
    y: Numpy array of target (label) data
        (if the model has a single output),
        or list of Numpy arrays (if the model has multiple outputs).
        If output layers in the model are nam

In [ ]:
import string
def string_vectorizer(sequence, alphabet=string.ascii_uppercase):
    vector = [[0 if char != letter else 1 for char in alphabet] 
                  for letter in sequence.upper()]
    return vector

regression_df['encoded_sequence'] = df0['sequence'].map(string_vectorizer)
regression_df.head()

In [ ]:
regression_df['sequence_length'] = regression_df['sequence'].map(len)
regression_df.head()